### Step 1. Recall the MDP and MRP Context ###

### MDP (Markov Decision Process):###
Defined as a tuple:

    (S,A,P,R,γ)
where:
S: Set of states
A: Set of actions
P: Transition probability function
R: Reward function
γ: Discount factor

### MRP (Markov Reward Process):###
Once a policy π is fixed, the MDP reduces to an MRP:
    (S,P^π,R^π,γ)

### State-Value Function:###
The value of a state under policy π:

    V^π(s)=E_π​[G_t​∣S_t​=s]
where the return is:
    G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots

In [ ]:
#Step 2. Environment Setup
from dataclasses import dataclass
import random

@dataclass(frozen=True)
class State:
    paddle_y: int
    ball_x: int
    ball_y: int
    vx: int
    vy: int

def step(state, action):
    """Transition function for simplified Pong."""
    paddle_y, ball_x, ball_y, vx, vy = state.paddle_y, state.ball_x, state.ball_y, state.vx, state.vy
    
    # Update paddle
    if action == "UP":
        paddle_y -= 1
    elif action == "DOWN":
        paddle_y += 1
    
    # Update ball
    ball_x += vx
    ball_y += vy
    
    # Bounce vertically
    if ball_y <= 0 or ball_y >= 4:
        vy *= -1
    
    # Reward
    reward = 0
    if ball_x == 0:  # ball reached left edge
        if abs(paddle_y - ball_y) <= 1:  # paddle hits
            reward = +1
            vx *= -1
        else:
            reward = -1
    
    return State(paddle_y, ball_x, ball_y, vx, vy), reward

In [2]:
# Step 3. Compute Discounted Return
def compute_return(rewards, gamma=0.99):
    G = 0
    for t, r in enumerate(rewards):
        G += (gamma ** t) * r
    return G

In [3]:
#Step 4. Generate an Episode
def random_policy(state):
    return random.choice(["UP", "DOWN", "STAY"])

def generate_episode(start_state, policy_fn, gamma=0.99, max_steps=20):
    """Simulate one episode."""
    states, actions, rewards = [], [], []
    s = start_state
    
    for _ in range(max_steps):
        a = policy_fn(s)
        ns, r = step(s, a)
        states.append(s)
        actions.append(a)
        rewards.append(r)
        s = ns
    
    # Compute returns (backward)
    returns = []
    G = 0
    for r in reversed(rewards):
        G = r + gamma * G
        returns.insert(0, G)
    
    return states, actions, rewards, returns

In [4]:
#Step 5. Estimate the State-Value Function
from collections import defaultdict

def estimate_v(num_episodes=500, gamma=0.99):
    V = defaultdict(float)
    counts = defaultdict(int)
    
    for _ in range(num_episodes):
        s0 = State(2, 4, 2, -1, 1)
        states, actions, rewards, returns = generate_episode(s0, random_policy, gamma)
        
        for s, G in zip(states, returns):
            V[s] += G
            counts[s] += 1
    
    # Average returns
    for s in V:
        V[s] /= counts[s]
    
    return V

In [5]:
#Step 6. Sample Run
# Estimate V(s) from many episodes
V = estimate_v(num_episodes=1000, gamma=0.99)

# Simulate one trajectory
s0 = State(2, 4, 2, -1, 1)
states, actions, rewards, returns = generate_episode(s0, random_policy, gamma=0.99, max_steps=5)

print("Sample Trajectory:")
for s, a, r, G in zip(states, actions, rewards, returns):
    print(f"State={s}, Action={a}, Reward={r}, Return={G:.2f}, V(s)≈{V[s]:.2f}")

Sample Trajectory:
State=State(paddle_y=2, ball_x=4, ball_y=2, vx=-1, vy=1), Action=DOWN, Reward=0, Return=-0.97, V(s)≈0.26
State=State(paddle_y=3, ball_x=3, ball_y=3, vx=-1, vy=1), Action=DOWN, Reward=0, Return=-0.98, V(s)≈0.09
State=State(paddle_y=4, ball_x=2, ball_y=4, vx=-1, vy=-1), Action=UP, Reward=0, Return=-0.99, V(s)≈-0.21
State=State(paddle_y=3, ball_x=1, ball_y=3, vx=-1, vy=-1), Action=DOWN, Reward=-1, Return=-1.00, V(s)≈0.29
State=State(paddle_y=4, ball_x=0, ball_y=2, vx=-1, vy=-1), Action=UP, Reward=0, Return=0.00, V(s)≈0.00


### Step 7. Talking Points ###

### State-Value Function V(s):###
    Represents the expected return from a given state under a policy.
    In Pong, states with the paddle near the ball generally have higher V(s).
    States far from the ball tend to have lower or negative V(s).

### Monte Carlo Estimation:###
    Averaging returns over many episodes approximates V(s).
    Requires multiple simulations since transitions are stochastic under a random policy.

### Sample Trajectory:###
    Shows how State, Action, Reward, Return, and Value link together.
    The discounted return explains why early rewards matter more than later ones.